<a href="https://colab.research.google.com/github/lazarussim12-beep/DLI_GRP_Assignment/blob/Ahmed/Ahmed%20Mohammed%20Khaled%20individual%20code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === REPRODUCIBILITY: Fix all random seeds ===
os.environ['PYTHONHASHSEED'] = '42'
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

# Load dataset
csv_files = [f for f in os.listdir('enron_data') if f.endswith('.csv')]
df = pd.read_csv(f'enron_data/{csv_files[0]}')

print(f"Dataset loaded: {df.shape}")

# Data preprocessing
df['Message'] = df['Message'].astype(str).str.lower().apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))
df['Spam/Ham'] = df['Spam/Ham'].map({'ham': 0, 'spam': 1})
X = df['Message'].values
y = df['Spam/Ham'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Start timing
start_time = time.time()

# Tokenization for neural network
max_words = 5000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_len)
X_test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_len)

# Fast Neural Network
model = Sequential([
    Embedding(max_words, 32, input_length=max_len),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(
    X_train_seq, y_train,
    batch_size=256,
    epochs=3,
    validation_split=0.1,
    verbose=2
)

proba_nn = model.predict(X_test_seq, verbose=0).flatten()

# TF-IDF features
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 3))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Count Vectorizer for Bernoulli NB
count_vectorizer = CountVectorizer(max_features=3000, binary=True)
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

# Extra Trees Classifier
et = ExtraTreesClassifier(n_estimators=50, random_state=42, n_jobs=-1)
et.fit(X_train_tfidf, y_train)
proba_et = et.predict_proba(X_test_tfidf)[:, 1]

# SGD Classifier
sgd = SGDClassifier(loss='log_loss', random_state=42, max_iter=1000)
sgd.fit(X_train_tfidf, y_train)
sgd_scores = sgd.decision_function(X_test_tfidf)
proba_sgd = 1 / (1 + np.exp(-sgd_scores))

# Bernoulli Naive Bayes
bnb = BernoulliNB(alpha=1.0)
bnb.fit(X_train_count, y_train)
proba_bnb = bnb.predict_proba(X_test_count)[:, 1]

# Logistic Regression (additional model like your friend's code)
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train_tfidf, y_train)
proba_lr = lr.predict_proba(X_test_tfidf)[:, 1]

# Function to calculate model parameters count
def count_parameters(model, model_type='sklearn'):
    if model_type == 'keras':
        return model.count_params()
    elif hasattr(model, 'coef_'):
        return model.coef_.size + 1
    elif hasattr(model, 'n_features_in_'):
        return model.n_features_in_
    else:
        return "N/A"

# Function to measure inference time
def measure_inference_time(model, X_test_data, model_type='sklearn'):
    start_time_inf = time.time()
    if model_type == 'sklearn':
        model.predict(X_test_data)
    elif model_type == 'keras':
        model.predict(X_test_data, verbose=0)
    end_time_inf = time.time()
    return (end_time_inf - start_time_inf) * 1000

# Generate predictions for individual models
et_pred = (proba_et > 0.5).astype(int)
sgd_pred = (proba_sgd > 0.5).astype(int)
bnb_pred = (proba_bnb > 0.5).astype(int)
nn_pred = (proba_nn > 0.5).astype(int)
lr_pred = (proba_lr > 0.5).astype(int)

# Ensemble predictions
ensemble_soft_proba = (proba_et + proba_sgd + proba_bnb + proba_nn) / 4
ensemble_soft_pred = (ensemble_soft_proba > 0.5).astype(int)

# Calculate comprehensive metrics for all models
models_results = []

# Extra Trees
et_acc = accuracy_score(y_test, et_pred)
et_prec = precision_score(y_test, et_pred)
et_rec = recall_score(y_test, et_pred)
et_f1 = f1_score(y_test, et_pred)
et_params = count_parameters(et)
et_inference = measure_inference_time(et, X_test_tfidf, 'sklearn')
models_results.append(['Extra Trees', et_acc, et_prec, et_rec, et_f1, et_params, et_inference])

# SGD Classifier
sgd_acc = accuracy_score(y_test, sgd_pred)
sgd_prec = precision_score(y_test, sgd_pred)
sgd_rec = recall_score(y_test, sgd_pred)
sgd_f1 = f1_score(y_test, sgd_pred)
sgd_params = count_parameters(sgd)
sgd_inference = measure_inference_time(sgd, X_test_tfidf, 'sklearn')
models_results.append(['SGD Classifier', sgd_acc, sgd_prec, sgd_rec, sgd_f1, sgd_params, sgd_inference])

# Bernoulli Naive Bayes
bnb_acc = accuracy_score(y_test, bnb_pred)
bnb_prec = precision_score(y_test, bnb_pred)
bnb_rec = recall_score(y_test, bnb_pred)
bnb_f1 = f1_score(y_test, bnb_pred)
bnb_params = count_parameters(bnb)
bnb_inference = measure_inference_time(bnb, X_test_count, 'sklearn')
models_results.append(['Bernoulli Naive Bayes', bnb_acc, bnb_prec, bnb_rec, bnb_f1, bnb_params, bnb_inference])

# Fast Neural Network
nn_acc = accuracy_score(y_test, nn_pred)
nn_prec = precision_score(y_test, nn_pred)
nn_rec = recall_score(y_test, nn_pred)
nn_f1 = f1_score(y_test, nn_pred)
nn_params = count_parameters(model, 'keras')
nn_inference = measure_inference_time(model, X_test_seq, 'keras')
models_results.append(['Fast Neural Network', nn_acc, nn_prec, nn_rec, nn_f1, nn_params, nn_inference])

# Logistic Regression
lr_acc = accuracy_score(y_test, lr_pred)
lr_prec = precision_score(y_test, lr_pred)
lr_rec = recall_score(y_test, lr_pred)
lr_f1 = f1_score(y_test, lr_pred)
lr_params = count_parameters(lr)
lr_inference = measure_inference_time(lr, X_test_tfidf, 'sklearn')
models_results.append(['Logistic Regression (TF-IDF)', lr_acc, lr_prec, lr_rec, lr_f1, lr_params, lr_inference])

# Ensemble
ensemble_acc = accuracy_score(y_test, ensemble_soft_pred)
ensemble_prec = precision_score(y_test, ensemble_soft_pred)
ensemble_rec = recall_score(y_test, ensemble_soft_pred)
ensemble_f1 = f1_score(y_test, ensemble_soft_pred)
ensemble_params = "Combined"
ensemble_inference = et_inference + sgd_inference + bnb_inference + nn_inference
models_results.append(['Ensemble (ET+SGD+BNB+FastNN)', ensemble_acc, ensemble_prec, ensemble_rec, ensemble_f1, ensemble_params, ensemble_inference])

# Print comprehensive results table (like your friend's format)
print("\n" + "="*120)
print("MODEL EVALUATION TABLE: Accuracy, Precision, Recall, F1, Params, Inference Time")
print("="*120)
print(f"{'Model':<35} {'Accuracy':<10} {'Precision':<10} {'Recall':<10} {'F1':<10} {'Params':<10} {'Inference (ms)':<15}")
print("-"*120)

for model_name, acc, prec, rec, f1, params, inference in models_results:
    print(f"{model_name:<35} {acc:<10.2f} {prec:<10.2f} {rec:<10.2f} {f1:<10.2f} {str(params):<10} {inference:<15.2f}")

print("-"*120)
print("TF-IDF: Term Frequency-Inverse Document Frequency with N-grams (1-3)")
print("ET: Extra Trees Classifier")
print("SGD: Stochastic Gradient Descent Classifier")
print("BNB: Bernoulli Naive Bayes")
print("FastNN: Fast Neural Network with Global Average Pooling")
print("LR: Logistic Regression on TF-IDF features")
print("="*120)

time_taken = time.time() - start_time
minutes = int(time_taken // 60)
seconds = time_taken % 60
print(f"\nTotal runtime: {time_taken:.2f} seconds ({minutes} minutes {seconds:.2f} seconds)")

# Individual model performance
print(f"\nIndividual Model Accuracies:")
print(f"Extra Trees: {et_acc:.2f}")
print(f"SGD Classifier: {sgd_acc:.2f}")
print(f"Bernoulli NB: {bnb_acc:.2f}")
print(f"Fast Neural Net: {nn_acc:.2f}")
print(f"Logistic Regression: {lr_acc:.2f}")
print(f"ENSEMBLE: {ensemble_acc:.2f}")

# Final Accuracy Display
print("\n" + "="*50)
print("FINAL ENSEMBLE ACCURACY")
print("="*50)
print(f"Accuracy: {ensemble_acc:.2%}")
print(f"Accuracy: {ensemble_acc:.4f}")
print("="*50)

# === STORE RESULTS FOR AHMED CODE ===
y_test_ahmed = y_test
y_pred_ahmed = ensemble_soft_pred
y_proba_ahmed = ensemble_soft_proba
model_name_ahmed = "Ahmed_Ensemble"

def store_results(name, y_true, y_pred, y_proba, file_path="all_models_results.pkl"):
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
    results = {
        "Model": name,
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred),
        "Recall": recall_score(y_true, y_pred),
        "F1": f1_score(y_true, y_pred),
        "ROC_AUC": roc_auc_score(y_true, y_proba),
        "y_true": y_true,
        "y_pred": y_pred,
        "y_proba": y_proba
    }
    if os.path.exists(file_path):
        with open(file_path, "rb") as f:
            all_results = pickle.load(f)
    else:
        all_results = []
    all_results.append(results)
    with open(file_path, "wb") as f:
        pickle.dump(all_results, f)
    print(f"✅ Stored results for {name}")

store_results(model_name_ahmed, y_test_ahmed, y_pred_ahmed, y_proba_ahmed)

# ======== VISUALIZATION: Confusion Matrix, ROC, and Precision-Recall Curves =========
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, precision_recall_curve, auc

# Prepare data for ensemble visualization
y_true_ensemble = y_test
y_pred_ensemble = ensemble_soft_pred
y_proba_ensemble = ensemble_soft_proba

# 1. Confusion Matrix
plt.figure(figsize=(6, 5))
cm = confusion_matrix(y_true_ensemble, y_pred_ensemble)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Ham', 'Spam'])
disp.plot(cmap='Blues')
plt.title('Confusion Matrix - Ensemble Model')
plt.tight_layout()
plt.show()

# 2. ROC Curve
plt.figure(figsize=(8, 6))
fpr, tpr, _ = roc_curve(y_true_ensemble, y_proba_ensemble)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) - Ensemble Model')
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()

# 3. Precision-Recall Curve
plt.figure(figsize=(8, 6))
precision, recall, _ = precision_recall_curve(y_true_ensemble, y_proba_ensemble)
plt.plot(recall, precision, color='blue', lw=2)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - Ensemble Model')
plt.grid(True)
plt.tight_layout()
plt.show()